# Make PIK3CA hotspot table

In [1]:
import cptac
import cptac.utils as u
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import pandas.util.testing as tm

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load Cptac dfs

In [2]:
en = cptac.Endometrial()
col = cptac.Colon()
br = cptac.Brca()

In [6]:
#use get_somatic mutation not get genotype due to priority order for gen_genotype_all_vars
'''
This function takes a cptac cancer object and returns a df with the PIK3CA hotspot mutations counts.
@Param cptac_cancer: cptac cancer object
@Param cancer_name: string of cancer type for column label
'''
def makeHotspotTable(cptac_cancer, cancer_name):

    mutation_df = cptac_cancer.get_somatic_mutation() # get mutation data
    pik3ca = mutation_df.loc[mutation_df['Gene'] == "PIK3CA"] # only for PIK3CA
    pik3ca = pik3ca.replace(r'p.', "", regex = True)
    hotspots = pik3ca.groupby('Location').count()
    #get only hotspot mutations
    hotspot = hotspots[(hotspots.index.str.contains('E542K') | hotspots.index.str.contains('E545K') |hotspots.index.str.contains('H1047R'))]
    hotspot = hotspot.rename(columns={"Mutation": "Number of Mutations " + cancer_name})
    hotspot = hotspot[["Number of Mutations " + cancer_name]]
    return hotspot

Use fu

In [10]:
endo_df = makeHotspotTable(en, "Endometrial")
col_df = makeHotspotTable(col, "Colon")
br_df = makeHotspotTable(br, "Breast")

In [11]:
hotspot_table = endo_df.merge(col_df, on = "Location", how = "outer")
hotspot_table = hotspot_table.merge(br_df, on = "Location", how = "outer")
hotspot_table

Name,Number of Mutations Endometrial,Number of Mutations Colon,Number of Mutations Breast
Location,,,
E542K,4,1,1
E545K,9,5,4
H1047R,4,4,15


In [ ]:
hotspot_table.to_csv('PIK3CA_hotspot_table.csv')